In [2]:
import spectral.io.envi as envi
from proj.tasks import send_matrix_to_queue_task, create_sqlite3_conn_pool, NUM_PARTITION
import numpy as np
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
import os
import json
from datetime import datetime
import time

In [2]:
def read_image_spectral():
    return envi.open(
        "/home/ubuntu/code/agriculture-prediction/data/spectral_image/hyper_20220913_3cm.hdr", 
        "/home/ubuntu/code/agriculture-prediction/data/spectral_image/hyper_20220913_3cm.img"
    )

In [3]:
img = read_image_spectral()

In [4]:
NUM_ROW, NUM_COL, NUM_BAND = img.shape 

In [5]:
NUM_ROW

6478

In [6]:
NUM_COL

5287

In [7]:
RUN_ID = 10
TIME_RUN = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
CHUNK_READ_ROW = NUM_ROW
CHUNK_READ_COLUMN = 1000

In [8]:
def get_start_end_col(checkpoint_path, chunk):
    if not os.path.exists(checkpoint_path):
        start_col = 0
        end_col = start_col + chunk
    else:
        with open(checkpoint_path, "r") as f:
            checkpoint_data = json.load(f)
            last_end_col = checkpoint_data.get("end_col", 0)
            
            start_col = last_end_col
            end_col = min(NUM_COL, start_col + chunk)
    
    if start_col == NUM_COL:
        raise Exception("Already processed all columns")
    
    return start_col, end_col


In [9]:
def save_checkpoint(checkpoint_path, start_col, end_col):
    with open(checkpoint_path, "w") as f:
        checkpoint_data = {
            "start_col": start_col,
            "end_col": end_col
        }
        json.dump(checkpoint_data, f, indent=4)

In [10]:
CHECKPOINT_PATH = './checkpoint/checkpoint_run_nito_0.json'

In [11]:
start_col, end_col = get_start_end_col(CHECKPOINT_PATH, CHUNK_READ_COLUMN)

In [12]:
start_col

5000

In [13]:
end_col

5287

In [14]:

table_name = f"nito_{RUN_ID}"
if start_col == 0:
    sqlite3_conn_pool = create_sqlite3_conn_pool(NUM_PARTITION)
    for partition in range(NUM_PARTITION):
        # Connect to the SQLite database
        conn = sqlite3_conn_pool[partition]

        # Create a cursor object to execute SQL commands
        cursor = conn.cursor()

        # Execute the SQL command to drop the table
        cursor.execute(f"DROP TABLE IF EXISTS {table_name};")

        # Commit the transaction
        conn.commit()

        conn.close()
    
        print(f"Sucess drop table {table_name}")
else:
    print(f"Because start_col = {start_col} > 0, dont drop table {table_name}")

Because start_col = 5000 > 0, dont drop table nito_9


In [15]:
for col in range(start_col, end_col):
    if col % 100 == 0:
        print(f"col = {col}")
    send_matrix_to_queue_task.delay(RUN_ID, col, CHUNK_READ_ROW)

col = 5000
col = 5100


col = 5200


In [16]:
save_checkpoint(CHECKPOINT_PATH, start_col, end_col)